In [1]:
import numpy as np
import arcpy
from arcpy import sa
from arcpy.sa import *

In [ ]:
## Maps for quantiles 5% and 95% 
arcpy.CheckOutExtension("Spatial")

nee_layer = arcpy.Raster("_/gcas_bio_opt__.tif") # NPP data for each year
mask_raster = arcpy.Raster("_/ecoregions.tif") # mask of ecoregions
arcpy.env.cellSize = "MINOF"
nee_layer = sa.ExtractByMask(nee_layer, mask_raster)
nee_array = arcpy.RasterToNumPyArray(nee_layer)

# Exclude NoData
no_data_value = nee_layer.noDataValue
print(f"NoData: {no_data_value}")

if no_data_value is not None and np.isfinite(no_data_value):
    mask = (nee_array != no_data_value)
else:
    mask = np.ones_like(nee_array, dtype=bool)
   
nee_array_masked = np.where(mask, nee_array, np.nan)
print(f"Number of elements after masking: {np.count_nonzero(~np.isnan(nee_array_masked))}")

if np.isnan(np.nanmin(nee_array_masked)):
    raise ValueError("Data array includes only NaN values")

min_value = np.nanmin(nee_array_masked)
max_value = np.nanmax(nee_array_masked)
print(f"Min value: {min_value}, Max value: {max_value}")
q5 = np.nanpercentile(nee_array_masked, 5)
q95 = np.nanpercentile(nee_array_masked, 95)
print(f"5 quantile: {q5}, 95 quantile: {q95}")

if q5 >= q95:
    raise ValueError(f"5 qiantile is equal to or greater than 95 quantile: {q5} >= {q95}")

remap = RemapRange([
    [float(min_value), float(q5 + 1e-6), 1], # 5% to 1 class
    [float(q5 + 1e-6), float(q95), 2],       # 2 class - 5-95%
    [float(q95), float(max_value), 3]        # 3 class - 95%
])

quantile_nee2019 = Reclassify(nee_layer, "VALUE", remap, "NODATA")
final_raster_path = "_/quantile_nee_.tif"
quantile_nee2019.save(final_raster_path)

arcpy.CheckInExtension("Spatial")
arcpy.Delete_management(nee_layer)